# --- Day 8: Playground ---
https://adventofcode.com/2025/day/8

## Parse the Input Data

In [32]:
import numpy as np

In [65]:
def parse(filename):
    """Parse puzzle input data."""
    with open(f'../inputs/{filename}.txt') as f:
        return [
            np.array(tuple(map(int, line.strip().split(','))))
            for line in f.readlines()
        ]

In [173]:
parse("day-08_test")[:3]

[array([162, 817, 812]), array([ 57, 618,  57]), array([906, 360, 560])]

## Part 1
---

In [ ]:
from itertools import combinations
from math import prod

In [168]:
def make_circuts(pairs):

    while True:
        any_matches = False

        for i in range(len(pairs)):
            for j in range(len(pairs)):
                if i < j and pairs[j] != set():
                    match = False
                    for p in pairs[i]:
                        if p in pairs[j]:
                            any_matches = True
                            match = True
                            break
                    if match:
                        pairs[i] = pairs[i].union(pairs[j])
                        pairs[j] = set()

        if any_matches:
            make_circuts(pairs)
        else:
            break

    return pairs

In [169]:
def solve(coords, n, t):
    d = {}

    for pair in combinations(coords, 2):
        k = np.linalg.norm(pair[0] - pair[1])
        v = ["".join([str(d) for d in p]) for p in pair]
        d[k] = v

    sorted_keys = sorted(d.keys())
    min_n_pairs = [set(d[sorted_keys[i]]) for i in range(n)]

    circuts = make_circuts(min_n_pairs)
    circut_lens = sorted([len(c) for c in circuts], reverse=True)

    return prod(circut_lens[:t])


### Run on Test Data

In [171]:
solve(parse("day-08_test"), 10, 3) == 40

True

### Run on Input Data

In [172]:
solve(parse("day-08"), 1000, 3)

127551

## Part 2
---

In [ ]:
def solve2():
    pass

### Run on Test Data

In [ ]:
solve2(parse("day-08_test")) == 0

### Run on Input Data

In [ ]:
solve2(parse("day-08"))